In [0]:
%scala
val df = spark.read.format("csv").option("Header","true").load("dbfs:/FileStore/tables/IPL_matches.csv")
df.createOrReplaceTempView("DEMO")

df: org.apache.spark.sql.DataFrame = [id: string, season: string ... 16 more fields]

In [0]:
%sql
CREATE TEMP VIEW teams AS
SELECT DISTINCT team1 AS team FROM DEMO

In [0]:
%sql
SELECT * FROM teams

team
Sunrisers Hyderabad
Chennai Super Kings
Rising Pune Supergiant
Deccan Chargers
Kochi Tuskers Kerala
Rajasthan Royals
Gujarat Lions
Royal Challengers Bangalore
Kolkata Knight Riders
Rising Pune Supergiants


In [0]:
%sql
SELECT * FROM DEMO
LIMIT 100

id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,null
2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,null
3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,null
4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,null
5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,null,null,null
6,2017,Hyderabad,2017-04-09,Gujarat Lions,Sunrisers Hyderabad,Sunrisers Hyderabad,field,normal,0,Sunrisers Hyderabad,0,9,Rashid Khan,"Rajiv Gandhi International Stadium, Uppal",A Deshmukh,NJ Llong,null
7,2017,Mumbai,2017-04-09,Kolkata Knight Riders,Mumbai Indians,Mumbai Indians,field,normal,0,Mumbai Indians,0,4,N Rana,Wankhede Stadium,Nitin Menon,CK Nandan,null
8,2017,Indore,2017-04-10,Royal Challengers Bangalore,Kings XI Punjab,Royal Challengers Bangalore,bat,normal,0,Kings XI Punjab,0,8,AR Patel,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,null
9,2017,Pune,2017-04-11,Delhi Daredevils,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Delhi Daredevils,97,0,SV Samson,Maharashtra Cricket Association Stadium,AY Dandekar,S Ravi,null
10,2017,Mumbai,2017-04-12,Sunrisers Hyderabad,Mumbai Indians,Mumbai Indians,field,normal,0,Mumbai Indians,0,4,JJ Bumrah,Wankhede Stadium,Nitin Menon,CK Nandan,null


In [0]:
%sql
/*1. bat or field to win per venue based on toss decision*/
CREATE TEMP VIEW win_venue AS
SELECT team1, team2, toss_decision, venue, winner
FROM DEMO
WHERE winner = team1 AND toss_decision = "bat" OR winner = team2 AND toss_decision = "field";

In [0]:
%sql
SELECT ROUND((COUNT(CASE WHEN toss_decision = "bat" THEN 1 END)/ (COUNT(CASE WHEN toss_decision = "bat" THEN 1 END) + COUNT(CASE WHEN toss_decision = "field" THEN 1 END)))*100,2) AS BAT_WIN_PERC,
ROUND(100-(COUNT(CASE WHEN toss_decision = "bat" THEN 1 END)/ (COUNT(CASE WHEN toss_decision = "bat" THEN 1 END) + COUNT(CASE WHEN toss_decision = "field" THEN 1 END)))*100,2) AS FIELD_WIN_PERC, venue FROM win_venue
GROUP BY venue

BAT_WIN_PERC,FIELD_WIN_PERC,venue
33.33,66.67,Dubai International Cricket Stadium
20.0,80.0,Himachal Pradesh Cricket Association Stadium
50.0,50.0,"Sardar Patel Stadium, Motera"
31.25,68.75,"Punjab Cricket Association Stadium, Mohali"
40.0,60.0,Barabati Stadium
0.0,100.0,"Punjab Cricket Association IS Bindra Stadium, Mohali"
50.0,50.0,Nehru Stadium
10.0,90.0,Maharashtra Cricket Association Stadium
35.29,64.71,Eden Gardens
50.0,50.0,OUTsurance Oval


In [0]:
%sql
/*2. Face to Face at home*/
CREATE TEMP VIEW face_to_face AS
SELECT team1, team2, (COUNT(CASE WHEN winner = team1 AND result <> 'tie' THEN 1 END)) AS win_team1, 
(COUNT(CASE WHEN winner = team2 AND result <> 'tie' THEN 1 END)) AS win_team2 FROM DEMO
GROUP BY team2,team1

In [0]:
%sql
CREATE TEMP VIEW ftf_v AS
SELECT team1 AS HOME_TEAM, CONCAT(win_team1,'-', win_team2) AS MATCH_STATS_AT_HOME,team2 AS AWAY_TEAM FROM face_to_face 

In [0]:
%sql
SELECT * FROM ftf_v

HOME_TEAM,MATCH_STATS_AT_HOME,AWAY_TEAM
Mumbai Indians,6-4,Rajasthan Royals
Kochi Tuskers Kerala,0-2,Royal Challengers Bangalore
Gujarat Lions,0-1,Royal Challengers Bangalore
Kolkata Knight Riders,5-2,Deccan Chargers
Kings XI Punjab,2-1,Deccan Chargers
Chennai Super Kings,5-3,Deccan Chargers
Kings XI Punjab,3-4,Chennai Super Kings
Royal Challengers Bangalore,4-0,Pune Warriors
Sunrisers Hyderabad,1-2,Chennai Super Kings
Rising Pune Supergiant,0-1,Kings XI Punjab


In [0]:
%sql
/*3. winning toss and made a good decision */
CREATE TEMP VIEW toss_decision AS
SELECT team1, team2, toss_winner, toss_decision, CASE WHEN toss_decision = "bat" AND win_by_runs <> 0 THEN "Right Decision" WHEN toss_decision = "field" AND win_by_wickets <> 0 THEN "Right Decision" ELSE "made wrong decision" END AS DECISION_STATUS, winner FROM DEMO

In [0]:
%sql
SELECT winner AS TEAM, COUNT(CASE WHEN DECISION_STATUS = "Right Decision" THEN 1 END) AS TIMES_WON_BY_RIGHT_DECISION , COUNT(*) AS TOTAL_TOSS_WON, Round((COUNT(CASE WHEN DECISION_STATUS = "Right Decision" THEN 1 END)/COUNT(*))*100.0,2) AS ACCURACY 
FROM toss_decision
WHERE winner IS NOT null
GROUP BY winner
ORDER BY ACCURACY DESC

TEAM,TIMES_WON_BY_RIGHT_DECISION,TOTAL_TOSS_WON,ACCURACY
Gujarat Lions,10,13,76.92
Kochi Tuskers Kerala,4,6,66.67
Deccan Chargers,19,29,65.52
Kolkata Knight Riders,44,77,57.14
Chennai Super Kings,43,79,54.43
Delhi Daredevils,33,62,53.23
Rajasthan Royals,33,63,52.38
Mumbai Indians,48,92,52.17
Rising Pune Supergiant,5,10,50.0
Royal Challengers Bangalore,34,73,46.58


In [0]:
%sql
/*4. Most valuable players*/
SELECT player_of_match AS PLAYER, COUNT(player_of_match) AS TIMES_PLAYED_WELL FROM DEMO
WHERE winner IS NOT null
GROUP BY player_of_match,winner
HAVING COUNT(player_of_match) > 5
ORDER BY winner

PLAYER,TIMES_PLAYED_WELL
MS Dhoni,11
MEK Hussey,10
SK Raina,12
RA Jadeja,7
V Sehwag,10
SE Marsh,9
AD Russell,6
RV Uthappa,6
G Gambhir,10
YK Pathan,7
